In [5]:
import os
import gc
# import cv2
from PIL import Image
import numpy as np

# 加载所有图片，共64467张


def load_data():
    # Return a new array of given shape and type, without initializing entries.
    data = np.empty((64467, 60, 80, 3), dtype='float32')
#     data_arr=[[[[]]]]
#     label = np.empty((64467, 16), dtype='string')
    label_arr = [["" for i in range(16)]for j in range(64467)]
    # os.listdir(filename)返回filename中所有文件的文件名列表
    imgs = os.listdir('cardata')
    num = len(imgs)
    print(num)
    for i in range(num):
        # PIL 的 open() 函数用于创建 PIL 图像对象
        img_0 = Image.open('cardata/'+imgs[i])
        img = img_0.resize((80, 60), Image.ANTIALIAS)
        # Convert the input to an array
        arr = np.asarray(img, dtype='float32')
        data[i, :, :, :] = arr

        label_arr[i] = imgs[i].split('_')[0:16]  # 这个是字符串类型需要编码
        if i % 100 == 0:
            print(i/646, "%")
        del arr
        del img
        del img_0
        gc.collect()
    return data, label_arr

# 加载图片和标签

在初始化data的时候，使用data = np.empty((64000, 240, 320, 3), dtype='float32')会导致内存不足。针对此问题，首先采用的解决方案是先声明一个四维数组arr，每读一张图就往数组arr里头写入一个三维数组img。最后把数组转换为numpyarray，实在转换不了就算了

经实验，上述方法不能解决问题，现在启用plan B，换数据集。换数据集不光是出于内存的考虑，而是当前数据集中没有国产车。最后的训练结果一定是不尽人意的

plan C 减小图片像素，利用此数据集进行训练。之后手动标注国产车进行迁移学习。

构造label的时候，由于有些标签是字符串类型，所以需要先存储到一个多维数组里，最后进行编码处理

In [6]:
data, label_arr = load_data()


64467
0.0 %
0.15479876160990713 %
0.30959752321981426 %
0.46439628482972134 %
0.6191950464396285 %
0.7739938080495357 %
0.9287925696594427 %
1.08359133126935 %
1.238390092879257 %
1.3931888544891642 %
1.5479876160990713 %
1.7027863777089782 %
1.8575851393188854 %
2.0123839009287927 %
2.1671826625387 %
2.321981424148607 %
2.476780185758514 %
2.6315789473684212 %
2.7863777089783284 %
2.9411764705882355 %
3.0959752321981426 %
3.2507739938080493 %
3.4055727554179565 %
3.5603715170278636 %
3.7151702786377707 %
3.869969040247678 %
4.024767801857585 %
4.179566563467493 %
4.3343653250774 %
4.489164086687307 %
4.643962848297214 %
4.798761609907121 %
4.953560371517028 %
5.108359133126935 %
5.2631578947368425 %
5.41795665634675 %
5.572755417956657 %
5.727554179566564 %
5.882352941176471 %
6.037151702786378 %
6.191950464396285 %
6.3467492260061915 %
6.501547987616099 %
6.656346749226006 %
6.811145510835913 %
6.96594427244582 %
7.120743034055727 %
7.275541795665634 %
7.430340557275541 %
7.585139318

62.693498452012385 %
62.84829721362229 %
63.0030959752322 %
63.1578947368421 %
63.31269349845201 %
63.46749226006192 %
63.62229102167183 %
63.77708978328173 %
63.93188854489164 %
64.08668730650155 %
64.24148606811146 %
64.39628482972137 %
64.55108359133126 %
64.70588235294117 %
64.86068111455108 %
65.015479876161 %
65.17027863777089 %
65.3250773993808 %
65.47987616099071 %
65.63467492260062 %
65.78947368421052 %
65.94427244582043 %
66.09907120743034 %
66.25386996904025 %
66.40866873065015 %
66.56346749226006 %
66.71826625386997 %
66.87306501547988 %
67.02786377708978 %
67.18266253869969 %
67.3374613003096 %
67.49226006191951 %
67.6470588235294 %
67.80185758513932 %
67.95665634674923 %
68.11145510835914 %
68.26625386996903 %
68.42105263157895 %
68.57585139318886 %
68.73065015479877 %
68.88544891640866 %
69.04024767801857 %
69.19504643962848 %
69.3498452012384 %
69.50464396284829 %
69.6594427244582 %
69.81424148606811 %
69.96904024767802 %
70.12383900928792 %
70.27863777089783 %
70.43343

In [8]:
label=np.array(label_arr)

In [11]:
# 保存到文件 避免下次重复计算
import scipy.io as io
data_mat_path = 'mats/'
label_mat_path = 'labels/'
io.savemat(data_mat_path, {'name': data})
io.savemat(label_mat_path, {'name': label})

# 从mat文件加载保存的图片和标签

In [40]:
# 加载data、label
data_filename = 'data.mat'
label_filename = 'label.mat'
data = io.loadmat(data_mat_path+data_filename)["name"]
label = io.loadmat(label_mat_path+label_filename)["name"]


In [193]:
label = io.loadmat(label_mat_path+label_filename)["name"]

In [194]:
for i in range(64467):  # 去除因对齐添加的空格
    for j in range(16):
        label[i][j] = label[i][j].strip()

# 进行数据清洗

In [195]:
# 选取所有label作为使用，因为这样的估价会更加准确
# 首先对label进行编码，数字类型的和字符串类型的要使用不同的编码方式
# 参考 https://blog.csdn.net/luguanyou/article/details/80598122

In [263]:
# 先将label转换为DataFrame的格式
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from pandas import DataFrame

data_frame_label = DataFrame(label, index=None, columns=['Make', 'Module', 'Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
                                                         'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
                                                         'Drivetrain', 'PassengerCpacity', 'PassengerDoors', 'BodyStyle'])


In [264]:
data_frame_label

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
0,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
1,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
2,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
3,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
4,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64462,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64463,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64464,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64465,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV


In [202]:
row,col=data_frame_label.shape

In [223]:
data_frame_label.loc[256:256]

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
256,Acura,MDX,2013,43,18,300,37,6,78,68,191,nan,AWD,7,4,SUV


In [226]:
# 这时候需要进行数据清洗，因为有空值的存在
# 先查看一下有多少记录存在空值

cols_list = ['Make', 'Module', 'Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
             'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
             'Drivetrain', 'PassengerCpacity', 'PassengerDoors', 'BodyStyle']
del_index = []
for cols in cols_list:
    for i in range(row):
        if data_frame_label[cols][i] == 'nan':
            del_index.append(i)
len(del_index)

24947

In [227]:
row

64467

In [256]:
# 可以看到有24947个记录存在nan
# 处理空值主要有两种方式：剔除、替换
# 在这里存在空值的记录占到了三分之一，用剔除法不太合适，所以决定进行均值填充
# 但是有四个字段是字符串类型，无法进行均值计算，我们选择保留它
import re

from fnmatch import fnmatchcase as match

num_cols_list = ['Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
                 'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
                 'PassengerCpacity', 'PassengerDoors']
# avg_year = avg_msrp = avg_fws = avg_horsr = avg_disp = avg_engine = avg_w = avg_h = avg_l = avg_gas = avg_passcap = avg_passdoor = 0
avg = [0 for i in range(len(num_cols_list))]
sums = [0 for i in range(len(num_cols_list))]
num = [0 for i in range(len(num_cols_list))]
for i in range(row):
    for j in range(len(num_cols_list)):
        x = data_frame_label[num_cols_list[j]][i]
#         if match(x,[0-9]\d*):
        if match(x,"[0-9]*"):
            num[j] += 1
            sums[j] += int(x)

In [257]:
num

[64467,
 63673,
 63732,
 63569,
 62257,
 62145,
 63609,
 63560,
 57776,
 57494,
 63740,
 63740]

In [258]:
sums

[129901542,
 3059783,
 1130543,
 17559500,
 1929296,
 339704,
 4660909,
 3928630,
 10685056,
 1214799,
 316266,
 223343]

In [259]:
for i in range(len(num_cols_list)):
    avg[i]=int(sums[i]/num[i])
avg

[2015, 48, 17, 276, 30, 5, 73, 61, 184, 21, 4, 3]

In [265]:
# 有了均值之后，对非法值进行修改
for i in range(row):
    for j in range(len(num_cols_list)):
        x = data_frame_label[num_cols_list[j]][i]
        if match(x, "[0-9]*"):
            data_frame_label[num_cols_list[j]][i]=int(x)
        else:
            data_frame_label[num_cols_list[j]][i]=avg[j]
#             print("修改了一处")


修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

...

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一处
修改了一

In [267]:

# 再查看一下有多少记录数值字段存在空值

del_index = []
for cols in num_cols_list:
    for i in range(row):
        if data_frame_label[cols][i] == 'nan':
            del_index.append(i)
len(del_index)

0

In [270]:
data_frame_label.head()

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
0,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
1,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
2,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
3,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
4,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr


In [271]:
data_frame_label.shape

(64467, 16)

# 进行独热编码

In [272]:
# 然后对label中的不同标签分别使用sklearn的LabelBinarizer进行编码，替换原先label的标签
encoder = LabelBinarizer()

make_onehot = encoder.fit_transform(data_frame_label['Make'])
module_onehot = encoder.fit_transform(data_frame_label['Module'])
drivetrain_onehot = encoder.fit_transform(data_frame_label['Drivetrain'])
bodystyle_one_hot = encoder.fit_transform(data_frame_label['BodyStyle'])

In [273]:
# 现在已经将字符串类型的标签进行独热编码，放到四个变量当中。接下来，移除原dataframelabel中这四列
# 在移除之前，要先建立独热编码后的标签和之前的联系，用字典表示
dict_make = {}
dict_module = {}
dict_drivetrain = {}
dict_bodystyle = {}

# 要作为字典的key，应该将独热编码转换为string
n = len(make_onehot)  # 64467

str_make_one_hot = [''for i in range(n)]
str_module_one_hot = [''for i in range(n)]
str_drivetrain_onehot = [''for i in range(n)]
str_bodystyle_one_hot = [''for i in range(n)]
# tuple_make_one_hot = tuple(make_onehot)
# tuple_module_one_hot = tuple(module_onehot)
# tuple_drivetrain_onehot = tuple(drivetrain_onehot)
# tuple_bodystyle_one_hot = tuple(bodystyle_one_hot)


for i in range(n):
    for j in range(len(make_onehot[0])):
        str_make_one_hot[i] += str(make_onehot[i][j])
    for j in range(len(module_onehot[0])):
        str_module_one_hot[i] += str(module_onehot[i][j])
    for j in range(len(drivetrain_onehot[0])):
        str_drivetrain_onehot[i] += str(drivetrain_onehot[i][j])
    for j in range(len(bodystyle_one_hot[0])):
        str_bodystyle_one_hot[i] += str(bodystyle_one_hot[i][j])

    dict_make[str_make_one_hot[i]] = data_frame_label['Make'][i].strip()
    dict_module[str_module_one_hot[i]] = data_frame_label['Module'][i].strip()
    dict_drivetrain[str_drivetrain_onehot[i]
                    ] = data_frame_label['Drivetrain'][i].strip()
    dict_bodystyle[str_bodystyle_one_hot[i]
                   ] = data_frame_label['BodyStyle'][i].strip()

In [274]:
# 移除四列,保存在新变量data_frame_label_nums 中
data_frame_label_nums = data_frame_label.drop(
    ['Make', 'Module', 'Drivetrain', 'BodyStyle'], axis=1)

In [275]:
# 现在想办法将移除的四列重新以独热编码的格式插入到data_frame_label中去

# 查看data_frame_label_num的格式
data_frame_label_nums.head()

,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,PassengerCpacity,PassengerDoors
0,2013,28,16,110,15,4,70,55,179,39,5,4
1,2013,28,16,110,15,4,70,55,179,39,5,4
2,2013,28,16,110,15,4,70,55,179,39,5,4
3,2013,28,16,110,15,4,70,55,179,39,5,4
4,2013,28,16,110,15,4,70,55,179,39,5,4


In [276]:
print(type(data_frame_label_nums),type(make_onehot))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [277]:
# 可以看到一个是DataFrame类型，另外四个是ndarray类型，所以我们首先将类型统一
data_ndarray_label_nums=data_frame_label_nums.values

In [278]:
data_ndarray_label_nums

array([[2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       ...,
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4]], dtype=object)

In [285]:
type(data_frame_label_nums['Year'][277])

int

In [178]:
# #去空格，转为数值型
# for i in range(64467):  # 去除因对齐添加的空格
#     for j in range(12):
#         data_ndarray_label_nums[i][j] = int(data_ndarray_label_nums[i][j].strip())

In [287]:
# sum=0
# num=0
# for i in range(64467):
#     if data_frame_label_nums['GasMileage'][i]!='nan':
#         sum+=int(data_frame_label_nums['GasMileage'][i])
#         num+=1
# avg=int(sum/num)
# avg

In [286]:
# # 由于存在nan，无法转换为int，所以对nan进行替换，替换为平均数

# for i in range(64467):  
#     for j in range(12):
#         if data_ndarray_label_nums[i][j] != 'nan':
#             data_ndarray_label_nums[i][j] = int(data_ndarray_label_nums[i][j])
#         else:
#             data_ndarray_label_nums[i][j] = avg

In [307]:
# 格式统一之后，为了进行拼接，需要对shape统一
data_ndarray_label_nums.shape

(64467, 12)

In [289]:
make_onehot.shape

(64467, 42)

In [304]:
module_onehot.shape

(64467, 323)

In [305]:
drivetrain_onehot.shape

(64467, 5)

In [306]:
bodystyle_one_hot.shape

(64467, 9)

In [332]:
# 一种实现思路是将data_ndarray_label_nums进行升维，如将3变为[3]，这样就可以和其他四个独热编码同构(需要把其它四个也升三维），可以进行拼接
# 另一种实现思路是直接将5个array进行拼接，表头的含义通过字典进行查找

# 由于第一种方案更接近人的思想，我们优先考虑这种

In [308]:
data_ndarray_label_nums

array([[2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       ...,
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4]], dtype=object)

In [317]:
data_ndarray_label_nums=data_ndarray_label_nums.reshape(64467,12,1)

In [321]:
data_ndarray_label_nums.shape

(64467, 12, 1)

In [324]:
make_onehot=make_onehot.reshape(64467,1,-1)

In [327]:
make_onehot[0][0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [328]:
module_onehot=module_onehot.reshape(64467,1,-1)
drivetrain_onehot=drivetrain_onehot.reshape(64467,1,-1)
bodystyle_one_hot=bodystyle_one_hot.reshape(64467,1,-1)

In [336]:
bodystyle_one_hot[0]

array([[0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [335]:
data_ndarray_label_nums[0]

array([[2013],
       [28],
       [16],
       [110],
       [15],
       [4],
       [70],
       [55],
       [179],
       [39],
       [5],
       [4]], dtype=object)

In [333]:
data_ndarray_label_one_hot = np.c_[data_ndarray_label_nums, make_onehot]

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [342]:
# 看来第一种方案行不通，shape不同无法进行拼接

# 只好采取第二种方案
# 首先把这些数据reshape到原来的结构
data_ndarray_label_nums=data_ndarray_label_nums.reshape(64467,-1)
make_onehot=make_onehot.reshape(64467,-1)
module_onehot=module_onehot.reshape(64467,-1)
drivetrain_onehot=drivetrain_onehot.reshape(64467,-1)
bodystyle_one_hot=bodystyle_one_hot.reshape(64467,-1)

In [343]:
data_ndarray_label_one_hot = np.c_[
    data_ndarray_label_nums, make_onehot, module_onehot, drivetrain_onehot, bodystyle_one_hot]

In [344]:
data_ndarray_label_one_hot.shape

(64467, 391)

In [347]:
data_ndarray_label_one_hot

array([[2013, 28, 16, ..., 0, 0, 0],
       [2013, 28, 16, ..., 0, 0, 0],
       [2013, 28, 16, ..., 0, 0, 0],
       ...,
       [2020, 50, 19, ..., 0, 0, 0],
       [2020, 50, 19, ..., 0, 0, 0],
       [2020, 50, 19, ..., 0, 0, 0]], dtype=object)

## 这里得到label

In [349]:
row, width_make_one_hot = make_onehot.shape
row, width_module_one_hot = module_onehot.shape
row, width_drivetrain_one_hot = drivetrain_onehot.shape
row, width_bodystyle_one_hot = bodystyle_one_hot.shape
print(width_make_one_hot, width_module_one_hot,
      width_drivetrain_one_hot, width_bodystyle_one_hot)


# 这里要记住data_ndarray_label_one_hot每一列的含义
# 0: 12  ['Year', 'MSRP', 'FrontWheelSize', 'Horsepower', 'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
#         'PassengerCpacity', 'PassengerDoors']

# 12:12+42 Make
# 12+42:12+42+323 Module
# 12+42+323: 12+42+323+5  DriveTrain
# 12+42+323+5:    BodyStyle

42 323 5 9


In [28]:
# imgs = os.listdir('cardata')
# img = Image.open('cardata/'+imgs[0])

In [301]:
# a=imgs[0].split('_')[0:16]
# print(a)
# make 制造商                   0     +
# module 款式                   1     +
# year 出场年份                 2  ?
# MSRP  价格/千元               3     +
# FrontWheelSize 前轮尺寸       4  -
# Horsepower 马力               5     +
# Displacement 油箱             6     +
# EngineType 引擎种类           7  -
# Width  宽度                   8     +
# Heigh  高度                   9     +
# Length 长度                   10    +
# GasMileage 油耗               11    +
# Drivetrain 传动系统           12  -
# PassengerCpacity 载客人数     13    +
# Passenger Doors  车门数量     14    +
# Body Style  车身风格          15  -

# 进行训练

In [352]:
# 到此为止，所有的图片保存在了data中，所有的标签保存到了data_ndarray_label_one_hot
# 标签名字过长，我们进行简化
import copy
label=copy.deepcopy(data_ndarray_label_one_hot)

In [353]:
label.shape

(64467, 391)

## 打乱顺序

In [354]:
#打乱顺序
#读取data矩阵的第一维数（图片的个数）
num_example=data.shape[0]
#产生一个num_example范围，步长为1的序列
arr=np.arange(num_example)
#调用函数，打乱顺序
np.random.shuffle(arr)
#按照打乱的顺序，重新排序
data=data[arr]
label=label[arr]


## 划分训练集、验证集、测试集

In [356]:
r_train = 0.6
r_test = 0.8

s_train = np.int(num_example * r_train)
s_test = np.int(num_example * r_test)

X_train = data[:s_train]
y_train = label[:s_train]

X_test = data[s_train:s_test]
y_test = label[s_train:s_test]

X_verify = data[s_test:]
y_verify = label[s_test:]

In [357]:
X_train.shape

(38680, 60, 80, 3)

## 构建网络

In [359]:

import tensorflow as tf


class Model(object):
    def __init__(self, batch_size=128, learning_rate=1e-3, num_labels=10):
        self._batch_size = batch_size
        self._learning_rate = learning_rate
        self._num_labels = num_labels

    def inference(self, images, keep_prob):
        with tf.variable_scope('conv1') as scope:
            kernel = self._create_weights([5, 5, 1, 32])
            conv = self._create_conv2d(images, kernel)
            bias = self._create_bias([32])
            preactivation = tf.nn.bias_add(conv, bias)
            conv1 = tf.nn.relu(preactivation, name=scope.name)
            self._activation_summary(conv1)

        # pool 1
        h_pool1 = self._create_max_pool_2x2(conv1)

        with tf.variable_scope('conv2') as scope:
            kernel = self._create_weights([5, 5, 32, 64])
            conv = self._create_conv2d(h_pool1, kernel)
            bias = self._create_bias([64])
            preactivation = tf.nn.bias_add(conv, bias)
            conv2 = tf.nn.relu(preactivation, name=scope.name)
            self._activation_summary(conv2)

        # pool 2
        h_pool2 = self._create_max_pool_2x2(conv2)

        with tf.variable_scope('local1') as scope:
            reshape = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
            W_fc1 = self._create_weights([7 * 7 * 64, 1024])
            b_fc1 = self._create_bias([1024])
            local1 = tf.nn.relu(tf.matmul(reshape, W_fc1) + b_fc1, name=scope.name)
            self._activation_summary(local1)

        with tf.variable_scope('local2_linear') as scope:
            W_fc2 = self._create_weights([1024, self._num_labels])
            b_fc2 = self._create_bias([self._num_labels])
            local1_drop = tf.nn.dropout(local1, keep_prob)
            local2 = tf.nn.bias_add(tf.matmul(local1_drop, W_fc2), b_fc2, name=scope.name)
            self._activation_summary(local2)
        return local2

    def train(self, loss, global_step):
        tf.summary.scalar('learning_rate', self._learning_rate)
        train_op = tf.train.AdamOptimizer(self._learning_rate).minimize(loss, global_step=global_step)
        return train_op

    def loss(self, logits, labels):
        with tf.variable_scope('loss') as scope:
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            cost = tf.reduce_mean(cross_entropy, name=scope.name)
            tf.summary.scalar('cost', cost)

        return cost

    def accuracy(self, logits, labels):
        with tf.variable_scope('accuracy') as scope:
            accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1)), 
            dtype=tf.float32), name=scope.name)
            tf.summary.scalar('accuracy', accuracy)
        return accuracy

    def _create_conv2d(self, x, W):
        return tf.nn.conv2d(input=x,
                            filter=W,
                            strides=[1, 1, 1, 1],
                            padding='SAME')

    def _create_max_pool_2x2(self, input):
        return tf.nn.max_pool(value=input,
                              ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1],
                              padding='SAME')

    def _create_weights(self, shape):
        return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1, dtype=tf.float32))

    def _create_bias(self, shape):
        return tf.Variable(tf.constant(1., shape=shape, dtype=tf.float32))

    def _activation_summary(self, x):
        tensor_name = x.op.name
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))